# Extensión de la API de Pandas

In [3]:
import itertools
import pandas as pd
import upsetplot

In [4]:
try:
    del pd.DataFrame.missing
except AttributeError:
    pass

In [7]:
@pd.api.extensions.register_dataframe_accessor("missing")
class MissingMethods:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    # Esta función sirve para contar la cantidad total de valores faltantes en el DataFrame.    
    def number_missing(self) -> int:
        return self._obj.isna().sum().sum()
    # Esta función calcula el número de valores completos restando el número de valores faltantes del tamaño total del DataFrame.
    def number_complete(self) -> int:
        return self._obj.size - self._obj.missing.number_missing()
        

    # Esta función proporciona un resumen de los valores faltantes a nivel de variable en el DataFrame.
    def missing_variable_summary(self) -> pd.DataFrame:
        # Crea un DataFrame booleano donde True indica un valor faltante y False indica un valor presente.
        is_missing = self._obj.isnull()
        
        # Utiliza el método pipe para encadenar operaciones en el DataFrame booleano.
        return is_missing.pipe(
            lambda df_1: (
                # Suma a lo largo de las filas para cada columna, contando los valores faltantes.
                df_1.sum()
                # Reinicia el índice y nombra la columna de recuento de valores faltantes.
                .reset_index(name="n_missing")
                # Renombra la columna del índice como "variable".
                .rename(columns={"index": "variable"})
                # Agrega nuevas columnas al DataFrame resultante.
                .assign(
                    # Agrega la columna "n_cases" que representa el número total de casos (filas).
                    n_cases=len(df_1),
                    # Agrega la columna "pct_missing" que representa el porcentaje de valores faltantes en cada variable.
                    pct_missing=lambda df_2: df_2.n_missing / df_2.n_cases * 100,
                )
            )
            
            
        )
      # Esta función proporciona un resumen de los valores faltantes a nivel de caso en el DataFrame.
    def missing_case_summary(self) -> pd.DataFrame:
        # Crea una nueva columna "case" que representa el índice del DataFrame.
        # Esta columna representa cada caso en el DataFrame.
        # La función lambda toma el DataFrame como entrada y devuelve el índice.
        return self._obj.assign(
            case=lambda df: df.index,
            # Crea una nueva columna "n_missing" que cuenta la cantidad de valores faltantes en cada caso.
            n_missing=lambda df: df.apply(
                axis="columns", func=lambda row: row.isna().sum()
            ),
            # Crea una nueva columna "pct_missing" que calcula el porcentaje de valores faltantes en cada caso.
            # Divide el recuento de valores faltantes entre el número total de columnas y multiplica por 100.
            pct_missing=lambda df: df["n_missing"] / df.shape[1] * 100,
        ).sort_values(by="pct_missing", ascending=False)[
            # Selecciona las columnas relevantes del DataFrame resultante.
            ["case", "n_missing", "pct_missing"]
        ]

    # Esta función genera una tabla que resume la cantidad de valores faltantes por variable.
    def missing_variable_table(self) -> pd.DataFrame:
        # Utiliza la función missing_variable_summary para obtener un resumen de valores faltantes por variable.
        return (
            self._obj.missing.missing_variable_summary()
            # Calcula la cantidad de variables con cierto número de valores faltantes y organiza los resultados.
            .value_counts("n_missing")
            # Reinicia el índice y renombra las columnas.
            .reset_index(name="n_variables")
            .rename(columns={"n_missing": "n_missing_in_variable"})
            # Agrega una columna "pct_variables" que representa el porcentaje de variables con valores faltantes.
            .assign(
                pct_variables=lambda df: df.n_variables / df.n_variables.sum() * 100
            )
            # Ordena el DataFrame por el porcentaje de variables con valores faltantes en orden descendente.
            .sort_values("pct_variables", ascending=False)
        )

     # Esta función genera una tabla que resume la cantidad de valores faltantes por caso.
    def missing_case_table(self) -> pd.DataFrame:
        # Utiliza la función missing_case_summary para obtener un resumen de valores faltantes por caso.
        return (
            self._obj.missing.missing_case_summary()
            # Calcula la cantidad de casos con cierto número de valores faltantes y organiza los resultados.
            .value_counts("n_missing")
            # Reinicia el índice y renombra las columnas.
            .reset_index(name="n_cases")
            .rename(columns={"n_missing": "n_missing_in_case"})
            # Agrega una columna "pct_case" que representa el porcentaje de casos con valores faltantes.
            .assign(pct_case=lambda df: df.n_cases / df.n_cases.sum() * 100)
            # Ordena el DataFrame por el porcentaje de casos con valores faltantes en orden descendente.
            .sort_values("pct_case", ascending=False)
        )

    # Esta función proporciona un resumen de los valores faltantes por cada intervalo de un número específico de filas.
    def missing_variable_span(self, variable: str, span_every: int) -> pd.DataFrame:
        # Crea una nueva columna "span_counter" que representa el número de intervalo para cada fila.
        # Utiliza la función np.repeat para repetir secuencialmente valores de 0 a span_every.
        # Luego, se limita la cantidad de repeticiones al número de filas del DataFrame original.
        return (
            self._obj.assign(
                span_counter=lambda df: (
                    np.repeat(a=range(df.shape[0]), repeats=span_every)[: df.shape[0]]
                )
            )
            # Agrupa el DataFrame por el contador de intervalo y realiza agregaciones.
            .groupby("span_counter")
            .aggregate(
                # Agrega la columna "n_in_span" que representa el tamaño del intervalo.
                n_in_span=(variable, "size"),
                # Agrega la columna "n_missing" que cuenta la cantidad de valores faltantes en el intervalo.
                n_missing=(variable, lambda s: s.isnull().sum()),
            )
            # Agrega nuevas columnas al DataFrame resultante.
            .assign(
                # Agrega la columna "n_complete" que representa la cantidad de valores completos en el intervalo.
                n_complete=lambda df: df.n_in_span - df.n_missing,
                # Agrega la columna "pct_missing" que representa el porcentaje de valores faltantes en el intervalo.
                pct_missing=lambda df: df.n_missing / df.n_in_span * 100,
                # Agrega la columna "pct_complete" que representa el porcentaje de valores completos en el intervalo.
                pct_complete=lambda df: 100 - df.pct_missing,
            )
            # Elimina la columna "n_in_span" del DataFrame resultante.
            .drop(columns=["n_in_span"])
            # Reinicia el índice del DataFrame resultante.
            .reset_index()
        )

   # Esta función proporciona un resumen de las secuencias de valores faltantes o completos en una variable.
    def missing_variable_run(self, variable) -> pd.DataFrame:
        # Utiliza la función itertools.groupby para agrupar los valores de la variable por su estado de presencia (faltante o completo).
        # La función lambda crea una lista con la longitud de cada grupo y su estado.
        rle_list = self._obj[variable].pipe(
            lambda s: [[len(list(g)), k] for k, g in itertools.groupby(s.isnull())]
        )

        # Crea un DataFrame a partir de la lista de longitud de carrera (run length) y estado de presencia.
        # Renombra las columnas a "run_length" y "is_na" y reemplaza True con "missing" y False con "complete".
        return pd.DataFrame(data=rle_list, columns=["run_length", "is_na"]).replace(
            {False: "complete", True: "missing"}
        )

    def sort_variables_by_missingness(self, ascending = False):

        return (
            self._obj
            .pipe(
                lambda df: (
                    df[df.isna().sum().sort_values(ascending = ascending).index]
                )
            )
        )

    def create_shadow_matrix(
        self,
        true_string: str = "Missing",
        false_string: str = "Not Missing",
        only_missing: bool = False,
    ) -> pd.DataFrame:
        return (
            self._obj
            .isna()
            .pipe(lambda df: df[df.columns[df.any()]] if only_missing else df)
            .replace({False: false_string, True: true_string})
            .add_suffix("_NA")
        )

    def bind_shadow_matrix(
        self,
        true_string: str = "Missing",
        false_string: str = "Not Missing",
        only_missing: bool = False,
    ) -> pd.DataFrame:
        return pd.concat(
            objs=[
                self._obj,
                self._obj.missing.create_shadow_matrix(
                    true_string=true_string,
                    false_string=false_string,
                    only_missing=only_missing
                )
            ],
            axis="columns"
        )

    def missing_scan_count(self, search) -> pd.DataFrame:
        return (
            self._obj.apply(axis="rows", func=lambda column: column.isin(search))
            .sum()
            .reset_index()
            .rename(columns={"index": "variable", 0: "n"})
            .assign(original_type=self._obj.dtypes.reset_index()[0])
        )

    # Plotting functions ---
    # esta funcion muestra el numero de valores faltantes por variable
    def missing_variable_plot(self):
        # Obtener el resumen de valores faltantes por variable y ordenarlo por el número de valores faltantes
        df = self._obj.missing.missing_variable_summary().sort_values("n_missing")

        # Crear un rango para las variables en el gráfico
        plot_range = range(1, len(df.index) + 1)

        # Dibujar líneas horizontales en la posición de cada variable
        plt.hlines(y=plot_range, xmin=0, xmax=df.n_missing, color="black")

        # Dibujar puntos para representar el número de valores faltantes en cada variable
        plt.plot(np.array(df.n_missing), plot_range, "o", color="black")

        # Etiquetar el eje y con el nombre de las variables
        plt.yticks(plot_range, df.variable)

        # Agregar una cuadrícula en el eje y
        plt.grid(axis="y")

        # Etiquetas para los ejes x e y
        plt.xlabel("Número de valores faltantes")
        plt.ylabel("Variable")
        

    # Valores faltantes en cada fila
    def missing_case_plot(self):
        # Obtener un resumen de la cantidad de valores faltantes por caso (fila)
        df = self._obj.missing.missing_case_summary()

        # Utilizar seaborn para crear un gráfico de distribución (histograma)
        sns.displot(data=df, x="n_missing", binwidth=1, color="black")

        # Agregar una cuadrícula en el eje x
        plt.grid(axis="x")

        # Etiquetar los ejes x e y
        plt.xlabel("Número de valores faltantes en el caso")
        plt.ylabel("Número de casos")

    def missing_variable_span_plot(
        self, variable: str, span_every: int, rot: int = 0, figsize=None
    ):

        (
            self._obj.missing.missing_variable_span(
                variable=variable, span_every=span_every
            ).plot.bar(
                x="span_counter",
                y=["pct_missing", "pct_complete"],
                stacked=True,
                width=1,
                color=["black", "lightgray"],
                rot=rot,
                figsize=figsize,
            )
        )

        plt.xlabel("Span number")
        plt.ylabel("Percentage missing")
        plt.legend(["Missing", "Present"])
        plt.title(
            f"Percentage of missing values\nOver a repeating span of { span_every } ",
            loc="left",
        )
        plt.grid(False)
        plt.margins(0)
        plt.tight_layout(pad=0)

    def missing_upsetplot(self, variables: list[str] = None, **kwargs):

        if variables is None:
            variables = self._obj.columns.tolist()



        return (
            self._obj.isna()
            .value_counts(variables)
            .pipe(lambda df: upsetplot.plot(df, **kwargs))
        )
        #----------------------------------------------------------------------------------
    def column_fill_with_dummies(self, column: pd.Series, proportion_below: float=0.10, jitter: float=0.07, seed: int=42) -> pd.Series:

        column = column.copy(deep= True)

        #Extract values metada
        missing_maks = column.isna()
        number_missing_values = missing_maks.sum()
        column_range = column.max() - column.min()

        #Shift data
        columns_shift = column.min() - column.min() * proportion_below
        
        #Create the "jitter" (noise) to be added around the point
        np.random.seed(seed)
        column_jitter = (np.random.rand(number_missing_values) - 2) * column_range * jitter

        #Save new dummy data
        column[missing_maks] = columns_shift + column_jitter

        return column

    def grafica_dos_variables(self,column_1, column_2):
        (
        self._obj.select_dtypes(exclude="category")
        .pipe(
            lambda df: df[df.columns[df.isna().any()]] #seleccionar solo columnas que tengan valores faltantes
        )
        .missing.bind_shadow_matrix(true_string = True, false_string = False)
        .apply(
            lambda column: column if '_NA' in column.name else column_fill_with_dummies(column, proportion_below=0.05, jitter=0.075)
        )
        .assign(
            nullity = lambda df : df[column_1 + "_NA"] | df[column_2 + "_NA"]
        )
        .pipe(
            lambda df : (
                sns.scatterplot(
                    data = df,
                    x = column_1,
                    y = column_2,
                    hue= "nullity"
                )
            )
        )
        )
    #-----------------------------------------------------------------

    def missing_columns(self) ->pd.DataFrame:
        columns_missing_value = (
            self._obj
            .select_dtypes(exclude="category")
            .pipe(
                lambda df: df[df.columns[df.isnull().any()]]
            )
            .sum()
        
        ) 
        return pd.DataFrame(data= columns_missing_value, columns=["n_missing"]).sort_values(by="n_missing", ascending=False)

/var/folders/52/bzfm61sj2znfqg12w34hpj6c0000gp/T/ipykernel_51475/3265467984.py:2: UserWarning: registration of accessor <class '__main__.MissingMethods'> under name 'missing' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class MissingMethods:


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=694a3d08-7f18-421d-9e2f-c2820a79680e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>